In [1]:
# Import functions for calculations
from functions import *
dir_org = 'example-data/'
dir_project = 'example-data/processed'

First form the scattering frames, and add glycerol/iodixanol changes in the same .tiff file

In [3]:
pos = [1, 2, 3]
per = np.arange(1, 8, 1)

for i in pos:
    folder = f'01 Blank chip pos{i} 6mW-100ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    frames = crop_by_half(frames, 'r')
    I0 = np.mean(frames, axis = 0).astype(np.uint16)

    folder = f'03 Silica beads 100nm pos{i} 6mW-100ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    frames = crop_by_half(frames, 'r')
    I1 = np.mean(frames, axis = 0).astype(np.uint16)

    folder = f'05 5per biotin vesicles pos{i} 6mW-100ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    frames = crop_by_half(frames, 'r')
    I2 = np.mean(frames, axis = 0).astype(np.uint16)

    folder = f'06 TREV pos{i} 6mW-100ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    frames = crop_by_half(frames, 'r')
    I3 = np.mean(frames, axis = 0).astype(np.uint16)

    folder = f'10 After incubation pos{i} 6mW-100ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    frames = crop_by_half(frames, 'r')
    I4 = np.mean(frames, axis = 0).astype(np.uint16)

    io = np.empty((16, 2048, 1024)).astype(np.uint16)

    folder = f'16 After incubation pos{i} 6mW-100ms'
    io_frame = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    io_frame = crop_by_half(io_frame, 'r')
    io[0, :, :] = np.mean(io_frame, axis=0).astype(np.uint16)

    for j in range(0, len(per)):
            name = f'Glycerol {per[j]} pos{i} 6mW-100ms'
            io_frame = tf.imread((f'{dir_org}/{name}/{name}_MMStack_Pos0.ome.tif'))
            io_frame = crop_by_half(io_frame, 'r')
            io[j+1, :, :] = np.mean(io_frame, axis=0).astype(np.uint16)

            name = f'Optiprep {per[j]} pos{i} 6mW-100ms'
            io_frame = tf.imread((f'{dir_org}/{name}/{name}_MMStack_Pos0.ome.tif'))
            io_frame = crop_by_half(io_frame, 'r')
            io[j+9, :, :] = np.mean(io_frame, axis=0).astype(np.uint16)

    folder = f'Back to PBS pos{i} 6mW-100ms'
    io_frame = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    io_frame = crop_by_half(io_frame, 'r')
    io[8, :, :] = np.mean(io_frame, axis=0).astype(np.uint16)
    
    tf.imwrite(f'{dir_project}/Exp_particles_not_aligned_pos{i}.tif', \
        np.concatenate((np.stack((I0, I1, I2, I3, I4)), io), axis=0), photometric='minisblack')

Align the frames

In [4]:
pos = 1
file = f'Exp_particles_not_aligned_pos{pos}'
path = f'{dir_project}/{file}.tif'
frames = tf.imread(path)

crop = 100
template_frame = 1
template = make_template(frames, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/Exp_particles_aligned_pos{pos}.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

pos = 2
file = f'Exp_particles_not_aligned_pos{pos}'
path = f'{dir_project}/{file}.tif'
frames = tf.imread(path)

crop = 100
template_frame = 1
template = make_template(frames, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        if i == 4 or i == 5:
            sz = frames[i].shape
            tform = np.float32([[1,0,115],[0,1,0]])
            frames[i] = cv2.warpAffine(frames[i], tform, (sz[1], sz[0]), flags=cv2.INTER_LINEAR)
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/Exp_particles_aligned_pos{pos}.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

pos = 3
file = f'Exp_particles_not_aligned_pos{pos}'
path = f'{dir_project}/{file}.tif'
frames = tf.imread(path)

crop = 100
template_frame = 1
template = make_template(frames, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/Exp_particles_aligned_pos{pos}.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

Form the fluorescence frames

In [5]:
pos = [1, 2, 3]

for i in pos:

    folder = f'09 After incubation pos{i} 6mW-500ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    I0 = crop_by_half(frames, 'l').astype(np.uint16)

    folder = f'15 After incubation pos{i} 6mW-500ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    I1 = crop_by_half(frames, 'l').astype(np.uint16)

    folder = f'11 After incubation red pos{i} 1200mV-500ms'
    frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
    I2 = crop_by_half(frames, 'l').astype(np.uint16)

    tf.imwrite(f'{dir_project}/fl_not_aligned_pos{i}.tif', np.stack((I0, I1, I2)), photometric='minisblack')

Form the time-resolved antibody binding frames (scattering and fluorescence)

In [6]:
folder = f'Exp_particles_aligned_pos1.tif'
frames = tf.imread(f'{dir_project}/{folder}')
frame0 = frames[3]

folder = f'07 CD9 CD41 AB 1ugml binding 6mW-200ms'
frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
frames_sc = crop_by_half(frames, 'r').astype(np.uint16)
frames_fl = crop_by_half(frames, 'l').astype(np.uint16)
tf.imwrite(f'{dir_project}/AB_binding1_sc.tif', \
        np.concatenate(([frame0], frames_sc), axis=0), photometric='minisblack')
tf.imwrite(f'{dir_project}/AB_binding1_fl.tif', \
        np.concatenate(([frames_fl[-1]], frames_fl), axis=0), photometric='minisblack')

folder = f'13 Anti-biotin AB 5ugml binding 7min 6mW-200ms'
frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
frames_sc = crop_by_half(frames, 'r').astype(np.uint16)
frames_fl = crop_by_half(frames, 'l').astype(np.uint16)
tf.imwrite(f'{dir_project}/AB_binding2_sc.tif', \
        np.concatenate(([frame0], frames_sc), axis=0), photometric='minisblack')
tf.imwrite(f'{dir_project}/AB_binding2_fl.tif', \
        np.concatenate(([frames_fl[-1]], frames_fl), axis=0), photometric='minisblack')

<tifffile.TiffFile '07 CD9 CD41 AB …ack_Pos0.ome.tif'> MMStack series is missing files. Returning subset (23, 1, 1, 1) of (500, 1, 1, 1)
<tifffile.TiffFile '13 Anti-biotin …ack_Pos0.ome.tif'> MMStack series is missing files. Returning subset (22, 1, 1, 1) of (500, 1, 1, 1)


In [7]:
file = f'AB_binding1_sc'
path = f'{dir_project}/{file}.tif'
frames = tf.imread(path)

crop = 100
template_frame = 0
template = make_template(frames, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/AB_binding1_aligned_sc.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

file = f'AB_binding2_sc'
path = f'{dir_project}/{file}.tif'
frames = tf.imread(path)

crop = 100
template_frame = 0
template = make_template(frames, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/AB_binding2_aligned_sc.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

In [9]:
folder = f'Exp_particles_aligned_pos1.tif'
frames0 = tf.imread(f'{dir_project}/{folder}')

folder = f'02 Silica beads 100nm 5e9 2 min 6mW-100ms'
frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
frames = crop_by_half(frames, 'r')

crop = 100
template_frame = 1
template = make_template(frames0, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    if i != template_frame:
        frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/SNP_binding.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

<tifffile.TiffFile '02 Silica beads …ck_Pos0.ome.tif'> MMStack series is missing files. Returning subset (163, 1, 1, 1) of (500, 1, 1, 1)


In [10]:
folder = f'Exp_particles_aligned_pos1.tif'
frames0 = tf.imread(f'{dir_project}/{folder}')

folder = f'06 Binding TREVs at 1e10 8min 6mW-100ms'
frames = tf.imread((f'{dir_org}/{folder}/{folder}_MMStack_Pos0.ome.tif'))
frames = crop_by_half(frames, 'r')

crop = 100
template_frame = 2
template = make_template(frames0, template_frame, crop)
frames_out = frames.copy()
for i, frame in enumerate(frames):
    frames_out[i] = template_matching(frames[i], template, crop)

outFilename = f'{dir_project}/EV_binding.tif'
tf.imwrite(outFilename, frames_out, photometric='minisblack')

<tifffile.TiffFile '06 Binding TREVs…ck_Pos0.ome.tif'> MMStack series is missing files. Returning subset (446, 1, 1, 1) of (500, 1, 1, 1)
